# Graph
**Definition:**  
A graph is a set of vertices and a collection of edges that each connect a pair of vertices

## Basic implementation

In [1]:
def recurr(v):
    v -= 1
    print(v)
    if v == 0:
        print("!!!")
        return # return应该是结束方程执行并返回
    
    recurr(v)
    
recurr(3) # 顺次执行先前未执行的代码，想像成stack

a = {}
try:
    a[2]
except:
    print(1)

2
1
0
!!!
1


In [2]:
# 以某种格式存放graph并能够读取
# 数据结构是算法的基础, e.g. 好的图表征是DFS，BFS搜索算法的基础（路径问题）
class Node():
    def __init__(self,v):
        self.value=v
        self.next=None
        
class Queue():
    def __init__(self):
        self.root = None
    
    def put(self, v):
        if self.root == None:
            self.root = Node(v)
        
        self.root.next = Node(v)
    
    def get(self):
        if self.root == None:
            return None
        value = self.root.value
        self.root = self.root.next
        return value
    
    def empty(self):
        if self.root == None:
            return True
        else:
            return False

class Graph:
    def __init__(self):
        self.V = None 
        self.E = None
        self.edges = None # if want a adj of certain V, exame all eadges
        self.edgeTo = {} # used for tracking the path
        self.adj_dict = {} # track the adj list
    
    def adj(self, v):
        adj_list = []
        all_connect = []
        # 找出edges中含有v的tuples
        for connect in self.edges: 
            # determin if contains v
            if (v in connect):
                if (connect[0]==connect[1] and connect[0]==v):continue
                elif (connect[0]==v):adj_list.append(connect[1])
                else: adj_list.append(connect[0])
                all_connect.append(connect)

        return adj_list#, connected_edges
    
    def read_from_file(self, file):
        f = open(file, mode='r')
        def read_from_txt(txt):
            lines=[]
            while(True):
                l = f.readline().replace('\n','')
                if l == '':
                    return lines
                lines.append(l)
        
        inputs = read_from_txt(f)
        V = inputs[0]
        E = inputs[1]
        edges = []
        for line in inputs:
            if line == V or line == E: continue
            line = line.split(" ")
            line[0] = int(line[0])
            line[1] = int(line[1])
            edges.append(line)
        self.V = int(V)
        self.E = int(E)
        self.edges = edges
        f.close()
        
        # 这是用dict， 如何用linked list？为什么用？（遍历，插入简单，增加新edge）
        for v in range(self.V):
            self.adj_dict[v] = self.adj(v)
            
    def addEdge(self, v, w):
        self.adj_dict[v].append(w)
        self.adj_dict[w].append(v)
        
    def degree(self, v: int)->int:
        count = 0
        for _ in range(len((self.adj(v)))):
            count+=1
        return count
    
    def maxdegree(self)->int:
        max_degree = 0
        for v in range(self.V):
            if self.degree(v) >= max_degree: max_degree = self.degree(v)
        return max_degree
    
    
    def dfs(self, v):
        # 2E query operations and V mark operations, constant time proportional to E+V
        self.mark[v] = True
        for i in self.adj(v) :
            if not self.mark[i]:
                self.edgeTo[i] = v
                self.dfs(i)
                
    def path_To(self, root:int, target:int, method):
        self.root = None
        self.mark = {}
        for v in range(self.V):
            self.mark[v] = False
        # call dfs to get path from root to target
        if method == "bfs":self.bfs(root)
        elif method == "dfs":self.dfs(root)
            
        def pathTo(v):
            if not self.mark[v]:
                print("There is no path found!")
                return None
            if v == root:
                self.root = Node(v)
            try:
                pathTo(self.edgeTo[v])
            except:
                return
            add_new_node(self.root, v)
            
        pathTo(target)
        return self.root
    
    def bfs(self,s):
        import queue
        q = queue.Queue()
        self.mark[s] = True
        q.put(s)
        while (not q.empty()): # 直到q为空为止
            w = q.get()
            for i in self.adj(w):
                if not self.mark[i]:
                    self.edgeTo[i] = w
                    self.mark[i] = True
                    q.put(i)
     
    
def add_new_node(root,value):
    while(root.next is not None):
        root = root.next
        
    root.next = Node(value)
    
def traverse_root(root):
    while(root is not None):
        print(root.value)
        root = root.next       
    

G = Graph()
G.read_from_file("text.txt")

In [3]:
# 如何逆序打印 linked list？
root = Node(0)
add_new_node(root, 3)
def reverse_print_linked_list(root):
    if (root == None):
        return
    
    reverse_print_linked_list(root.next)
    print(root.value)
    
reverse_print_linked_list(root)

3
0


In [4]:
G.adj_dict

{0: [5, 1, 2, 6],
 1: [0],
 2: [0],
 3: [4, 5],
 4: [3, 6, 5],
 5: [0, 4, 3],
 6: [4, 0],
 7: [8],
 8: [7],
 9: [12, 10, 11],
 10: [9],
 11: [12, 9],
 12: [9, 11]}

In [5]:
print("vertices adjacent to 0: {}".format(G.adj(0)))
print("edges connected to 0: {}".format(G.degree(2)))
print("maximum degree: {}".format(G.maxdegree()))

vertices adjacent to 0: [5, 1, 2, 6]
edges connected to 0: 1
maximum degree: 4


In [6]:
r = G.path_To(root=4, target=0,method="dfs")
traverse_root(r)

4
3
5
0


In [7]:
r = G.path_To(root=4, target=0,method="bfs")
traverse_root(r)

4
6
0


## Connected Component

In [16]:
# build a new class whose constructor allows passing a Graph instance as arguement
# the instance of CC support queries about connectivity
class CC():
    def __init__(self, G: Graph):
        self.mark = {}
        for v in range(G.V):
            self.mark[v] = False
        self.id = {}
        self.count = 0
        self.G = G
        self._CC()
        
    def _CC(self):
        for v in range(self.G.V):
            if not self.mark[v]:
                self._dfs(v)
                self.count += 1
            
    def _dfs(self, v):
        self.mark[v] = True
        self.id[v] = self.count
        for i in self.G.adj(v) :
            if not self.mark[i]:
                self._dfs(i)
                
    def connected(self,v,w):
        return self.id[v] == self.id[w]
                
    def _id(self):
        return self.id
    
                
C = CC(G)
print(C._id())
print("Is 0 and 1 are connected? {}".format(C.connected(0,1)))
print("Total components: {}".format(C.count))
if C.count != G.V:
    print("This graph is NOT connected!")

{0: 0, 5: 0, 4: 0, 3: 0, 6: 0, 1: 0, 2: 0, 7: 1, 8: 1, 9: 2, 12: 2, 11: 2, 10: 2}
Is 0 and 1 are connected? True
Total components: 3
This graph is NOT connected!


In [24]:
# check if the given graph is bipartite
class TwoColor():
    def __init__(self, G: Graph):
        self.mark = {}
        for v in range(G.V):
            self.mark[v] = False
        self.color = {} # a dictionary of boolean
        self.G = G
        # 从某一个点开始搜索，譬如第一个点
        self.color[0] = True
        for v in range(G.V):
            if not self.mark[v]:
                self.color[v] = True
                self._dfs(v)
        
    def _dfs(self, v):
        self.mark[v] = True
        for i in self.G.adj(v):
            if not self.mark[i]:
                self.color[i] = not self.color[v]
                self._dfs(i)
            elif self.color[v] == self.color[i]:
                self.isBipartite = False
                
twoColor = TwoColor(G)
print("Is the given graph bipartite?: {}".format(twoColor.isBipartite))

Is the given graph bipartite?: False


## Cycle Finding

In [46]:
class Cycle():
    def __init__(self, G: Graph):
        self.mark = {}
        for v in range(G.V):
            self.mark[v] = False
        self.G = G
        
        for s in range(G.V):
            if not self.mark[s]: # 这个语句是因为可能存在多个components
                self.dfs(s,s)
        
    def dfs(self, u, w):
        self.mark[u] = True
        for v in self.G.adj(u):
            if not self.mark[v]:
                self.G.edgeTo[v] = u
                self.dfs(v, u)
            elif v == w: self.hasCycle = True
    
    
cycle = Cycle(G)
print("Graph has cycle?: {}".format(cycle.hasCycle))

Graph has cycle?: True


In [11]:
class Celsius:
    def __init__(self, tem =0):
        self.set_tem(tem)
        
    def to_fahrenheit(self):
        return (self.get_tem() *1.8) + 32
    
    def get_tem(self):
        return self._tem
    # 每次init时都会自动调用这个方法
    def set_tem(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        self._tem = value
        


class Celsius:
    def __init__(self, temp=0):
        self._temp = temp
    # 本质是get setter，用处是当需要对得到和返回变量进行特殊处理时，可以用   
    @property
    def temperature(self):
        print("Getting")
        return self._temp
        
    @temperature.setter
    def temperature(self, value):
        self._temp = value
        
C = Celsius(10)
print(C.temperature)
C.temperature = 25
print(C.temperature)

Getting
10
Getting
25
